dependencies </n>
Env :: Forecastinit
Python=3.11.5

Libraries

In [168]:
# General 
import pandas as pd
import numpy as np

# sklearn 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# skforecast 
from skforecast.datasets import load_demo_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

# ML Models 
from xgboost import XGBRegressor

Input

In [169]:
Historical_data_org = pd.read_csv('Input/Actual.csv')
TimeKey_org = pd.read_csv('Input/Time.csv')
Forecast_level = ['Version.[Version Name]', 'Channel.[Channel]', 'Account.[Account]',
       'PnL.[PnL]', 'Demand Domain.[Demand Domain]', 'Region.[Region]',
       'Location.[Location]', 'Time.[Planning Month]', 'Item.[Item]']
Time_column = ['Time.[Planning Month]']
Historical_data_column = ['Actual']
#Date should be in format of time key column, which is available in TimeKey file 
Time_key_column_name = "Time.[PlanningMonthKey]"
Time_key_date_format = "%m/%d/%Y %I:%M:%S %p"
Historic_start_date = "M09-20" 
Historic_end_date = "M12-22"
Forecast_start_date = "M01-23"
Forecast_end_date = "M03-23"
drivers = ['driver_Month']
models = ["Random Forest", "XG Boost"]


Custom Functions

In [170]:
def split(data,start_date_key,end_date_key,X_cols,y_col):
    # Data filter based on start and end date, then split in X and y based on provided columns 
    data = data[(data['TimeKey']>=start_date_key)&(data['TimeKey']<=end_date_key)]
    X = data[['key','TimeKey']+X_cols]
    y = data[[y_col]]
    return X, y

Processing Input

In [171]:
Forecast_level_minus_time = Forecast_level.copy()
Forecast_level_minus_time.remove(Time_column[0])
Forecast_level_minus_time

['Version.[Version Name]',
 'Channel.[Channel]',
 'Account.[Account]',
 'PnL.[PnL]',
 'Demand Domain.[Demand Domain]',
 'Region.[Region]',
 'Location.[Location]',
 'Item.[Item]']

Copy Input 

In [172]:
Historical_data = Historical_data_org.copy(deep=True)
TimeKey = TimeKey_org.copy(deep=True)
TimeKey['TimeKey'] = pd.to_datetime(TimeKey[Time_key_column_name], format=Time_key_date_format)

In [173]:
# calculating Historic_end_date_key
Historic_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_start_date]['TimeKey'].values[0])
Historic_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Historic_end_date]['TimeKey'].values[0])
Forecast_start_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_start_date]['TimeKey'].values[0])
Forecast_end_date_key = pd.to_datetime(TimeKey[TimeKey[Time_column[0]]==Forecast_end_date]['TimeKey'].values[0])

Processing Copy Input

In [174]:
Historical_data[Historical_data_column] = Historical_data[Historical_data_column].astype(float)


Creating Key in Historical Data

In [175]:
# creating keys 
Historical_data['key'] = Historical_data[Forecast_level_minus_time].astype(str).agg("__ MDJoinner__".join, axis=1)
# dropping columns, which is already present in keys 
Historical_data.drop(Forecast_level_minus_time,axis=1,inplace=True) 
Historical_data.head(2)

,Time.[Planning Month],Actual,key
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...


Custom Filter

In [176]:
Historical_data = Historical_data[Historical_data['key']=='CurrentWorkingView__ MDJoinner__B2B__ MDJoinner__AMO__ MDJoinner__DP__ MDJoinner__DP__ MDJoinner__ShipTo1__ MDJoinner__DP__ MDJoinner__Loctite 248 19g Stick']
# Historical_data['key'].values

Merging Time Key in Historical Data

In [177]:
Historical_data = pd.merge(Historical_data,TimeKey[Time_column+["TimeKey"]],on=Time_column,how='left')
Historical_data.head(2)

,Time.[Planning Month],Actual,key,TimeKey
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02


In [178]:
# Creating drivers 
Historical_data['driver_Month'] = Historical_data['TimeKey'].dt.month
Historical_data.head()

,Time.[Planning Month],Actual,key,TimeKey,driver_Month
0,M07-20,445.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-07-05,7
1,M08-20,711.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-02,8
2,M09-20,462.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-08-30,8
3,M10-20,174.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-10-04,10
4,M11-20,179.0,CurrentWorkingView__ MDJoinner__B2B__ MDJoinne...,2020-11-01,11


Train and Test Data

In [179]:
X_train, y_train = split(Historical_data, Historic_start_date_key,Historic_end_date_key,drivers,Historical_data_column[0])
X_test,y_test = split(Historical_data, Forecast_start_date_key, Forecast_end_date_key, drivers, Historical_data_column[0])

In [180]:
# # from sklearn.preprocessing import FunctionTransformer
# # transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

# forecaster = ForecasterAutoreg(
#                  regressor = RandomForestRegressor(random_state=123),
#                  lags      = [1]
#              )

# forecaster.fit(y=y_train[Historical_data_column[0]])
# forecaster

In [201]:
skforecastpredict(models,X_train,y_train,X_test)

In [200]:
def skforecastpredict(models,X_train, y_train, X_test):
    # Predicted output data frame 
    Output = pd.DataFrame()

    # converting data frame to series, as sk forecast y accept series 
    y_train_loc = y_train[y_train.columns.values[0]]

    # Random Forest ========================================================================================================================= 
    if "Random Forest" in models:
        randomforestforecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = [2]
             )
        randomforestforecaster.fit(y= y_train_loc)
        y_hat_randomforest = randomforestforecaster.predict_interval(steps=11).reset_index(drop=True)
        print(y_hat_randomforest)  
        y_hat_randomforest.columns = ['Random Forest Y_hat', 'Random Forest Lower Bound', 'Random Forest Upper Bound']
        Output = pd.concat([Output, y_hat_randomforest], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
    
    #XG Boost ================================================================================================================================
    if "XG Boost" in models:
        xgboostForecaster = ForecasterAutoreg(
                regressor= XGBRegressor(random_state = 213),
                lags= [1,2]
            )      
        xgboostForecaster.fit(y=y_train_loc)
        y_hat_xgboost = xgboostForecaster.predict_interval(steps=11).reset_index(drop=True)
        y_hat_xgboost.columns = ['XG Boost Y_hat', 'XG Boost Lower Bound', 'XG Boost Upper Bound']
        Output = pd.concat([Output, y_hat_xgboost], axis=1)
    # xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
        
    return Output

In [54]:
# Predict intervals
# predictions = 
forecaster.predict_interval(steps=1)
forecaster.predict_interval(steps=11)
# predictions.head(1)

/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
/home/mdbahauddinlinux/anaconda32023/envs/Forecastinit/lib/python3.11/site-packages/skforecast/utils/utils.py:877: UserWarning: `last_window` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


,pred,lower_bound,upper_bound
1,251.0400,182.140000,417.175500
2,230.6350,154.260500,541.190500
3,177.3795,143.727500,569.124250
4,229.8500,144.845500,590.690000
5,153.8595,160.200775,598.240000
6,191.9200,142.487000,612.869350
7,252.3400,143.407500,595.214750
8,230.6350,152.443225,583.800500
9,177.3795,160.534750,570.258750
10,229.8500,144.255475,671.191275


In [198]:
from sklearn.preprocessing import FunctionTransformer
transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 2
             )

forecaster.fit(y=y_train[Historical_data_column[0]])
forecaster

forecaster.predict_interval(steps=11)

,pred,lower_bound,upper_bound
2,242.180,171.91925,416.80000
3,275.090,206.50500,449.71000
4,261.680,120.64050,566.39950
5,539.310,146.78275,667.02900
6,244.500,153.08075,621.96000
7,452.380,131.19425,629.60000
8,225.340,138.79750,609.27000
9,221.270,137.12175,571.63900
10,215.460,141.00500,595.91575
11,202.755,139.73150,627.32000


In [199]:
from sklearn.preprocessing import FunctionTransformer
transformer_y = FunctionTransformer(func=np.log1p, inverse_func=np.expm1)

forecaster = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123),
                 lags      = 1
             )

forecaster.fit(y=y_train[Historical_data_column[0]])
forecaster

forecaster.predict_interval(steps=11)

,pred,lower_bound,upper_bound
1,251.0400,182.140000,417.175500
2,230.6350,154.260500,541.190500
3,177.3795,143.727500,569.124250
4,229.8500,144.845500,590.690000
5,153.8595,160.200775,598.240000
6,191.9200,142.487000,612.869350
7,252.3400,143.407500,595.214750
8,230.6350,152.443225,583.800500
9,177.3795,160.534750,570.258750
10,229.8500,144.255475,671.191275


In [133]:
y_train = y_train[y_train.columns.values[0]]

    # Random Forest 
# if "Random Forest" in models:
randomforestforecaster = ForecasterAutoreg(
            regressor = RandomForestRegressor(random_state=123),
            lags      = [2]
        )
print(y_train)
randomforestforecaster.fit(y= y_train)
y_hat_randomforest = forecaster.predict_interval(steps=11)        

2     462.0
3     174.0
4     179.0
5     231.0
6     126.0
7     212.0
8     231.0
9     135.0
10    182.0
11    288.0
12    431.0
13    693.0
14    473.0
15    148.0
16    188.0
17    241.0
18    228.0
19    246.0
20    209.0
21    167.0
22    175.0
23    232.0
24    540.0
25    732.0
26    627.0
27    133.0
28    153.0
29    194.0
Name: Actual, dtype: float64
